In [1]:
import os
import logging
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from textbrewer import GeneralDistiller, TrainingConfig, DistillationConfig
from datasets import load_dataset
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# 配置日志，设置日志级别为INFO，指定日志格式
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# 获取当前脚本文件的绝对路径
current_script_path = os.path.abspath('Distillation.ipynb')
logger.info(f"Current script path: {current_script_path}")

# 获取当前脚本文件所在的目录
current_script_dir = os.path.dirname(current_script_path)
logger.info(f"Current script directory: {current_script_dir}")


/root/sunhao11/guorui/HZQ/quantizaton/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/sunhao11/guorui/HZQ/quantizaton/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/root/sunhao11/guorui/HZQ/quantizaton/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/root/sunhao11/guorui/HZQ/quantizaton/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namesp

In [2]:
## 定义方法

def messages_to_prompt(messages, system_prompt='You are a helpful assistant.', fill_system_prompt=True):
    """
    将 messages 转换为 Qwen2 模型的输入 prompt。
    
    :param messages: 包含对话消息的列表，每个消息是一个字典，包含 'role' 和 'content' 字段。
                    例如: [{'role': 'system', 'content': 'You are a helpful assistant.'}, 
                          {'role': 'user', 'content': 'What is the capital of France?'}]
    :return: 转换后的 prompt 字符串，适用于 Qwen2 模型。
    """
    prompt = ""
    if fill_system_prompt and messages[0]['role'] != 'system':
        messages.insert(0, {'role': 'system', 'content': system_prompt})
    for message in messages:
        role = message['role']
        content = message['content']
        if role == 'system':
            prompt += f"<|im_start|>system\n{content}<|im_end|>\n"
        elif role == 'user':
            prompt += f"<|im_start|>user\n{content}<|im_end|>\n"
        elif role == 'assistant':
            prompt += f"<|im_start|>assistant\n{content}<|im_end|>\n"
        elif role == 'shipper':
            prompt += f"<|im_start|>shipper\n{content}<|im_end|>\n"
        elif role == 'knowledge':
            prompt += f"<|im_start|>knowledge\n{content}<|im_end|>\n"
        else:
            raise ValueError(f"Unknown role: {role}")
    
    # 添加 assistant 的开始标记，表示模型需要生成回复
    prompt += "<|im_start|>assistant\n"
    
    return prompt

In [3]:
# 加载教师模型（DeepSeek-R1:1.5B）
teacher_model_name = os.path.join("/root/matrix/LLM-Models-Export/huzhiqiang/", "Qwen2-1.5B-Instruct_20250326-车辆匹配OK")
logger.info(f"Loading teacher model: {teacher_model_name}")
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_name,
    local_files_only=True
)

teacher_model = AutoModelForCausalLM.from_pretrained(teacher_model_name,
    local_files_only=True
)


2025-04-24 18:15:03,977 - INFO - Loading teacher model: /root/matrix/LLM-Models-Export/huzhiqiang/Qwen2-1.5B-Instruct_20250326-车辆匹配OK


In [4]:
# 加载学生模型（Qwen）
student_model_name = os.path.join("/root/LLMmodels/qwen/", "Qwen2___5-1___5B-Instruct")  # 确保模型名称正确
logger.info(f"Loading student model: {student_model_name}")
student_tokenizer = AutoTokenizer.from_pretrained(student_model_name,
    local_files_only=True
)
student_model = AutoModelForCausalLM.from_pretrained(student_model_name,
    local_files_only=True
)


2025-04-24 18:15:04,735 - INFO - Loading student model: /root/LLMmodels/qwen/Qwen2___5-1___5B-Instruct


In [5]:
# 准备数据集

train_messages = json.load(open('/root/sunhao11/guorui/HZQ/prompt_data/车辆匹配sft.json', 'r',encoding='UTF-8'))
valid_messages = json.load(open('/root/sunhao11/guorui/HZQ/prompt_data/truck_check_sft_2.json', 'r',encoding='UTF-8'))
train_text = []
valid_text = []
for msg in train_messages:
    train_text.append(messages_to_prompt(msg).strip())

for msg in valid_messages:
    valid_text.append(messages_to_prompt(msg).strip())
    

# datasets_name = os.path.join(current_script_dir, "../models/Dataset/wikitext-2-raw/")  # 确保模型名称正确
# data_files = {
#     "train": datasets_name+"wiki.train.raw",
#     "test": datasets_name+"wiki.test.raw"
# }
# logger.info(f"Loading dataset from local files: {data_files}")
# dataset = load_dataset("text", data_files=data_files)
train_dataset = train_text
eval_dataset = valid_text
print(train_dataset[0])

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<任务>
在货运领域，货主发货后会有司机（也叫用户）前来咨询，你是货主（也叫老板）的助手，任务是帮货主回答司机问题，你的回复需结合多维度的信息
* [货源信息]和[备注]是货主发货时填写的要求，你的回复在提到货主要求时必须遵守二者，当二者信息有冲突时，默认以[备注]为准
* [历史对话]记录了过去你、货主和司机的对话，你的回复需要关注其中的有用信息。注意[历史对话]中你的回复是可能出错的，不要受到过去自己错误回复的影响
* [当前问题]是当前轮司机的问题，你必须理解清楚当前司机的意图再回复
* [回复指令]记录了货主对司机车辆各维度提供的回复指令，你的回复若涉及这些维度，就必须遵从对应的回复指令。当指令内部或与外部其他信息有冲突时，优先考虑[历史对话]中的货主回复，再考虑[备注]，其次是行业知识指令，最后才是其他维度的指令
* [通用规则]是指导你回复的一些通用性规则，优先级最低，但最通用
</任务>

<通用规则>
* [当前问题]涉及多个维度时，若其中一个维度需拒绝，只用这个维度拒绝就行
* [当前问题]中不涉及需拒绝的维度时，知道的就答，不知道的就说不知道（>=2个维度不知道时，可以简洁地表达为“其他的不知道”），不能忽略不答；若结合指令存在[当前问题]没提的需拒绝的维度，则进行提醒
</通用规则>

<货源信息>
货名：太阳能板；包装方式：托盘；托盘/吨包数量：None；装卸方式：None；单个重量：None；总重量：2.0-4.0吨；总体积：None；总高度：None；需要车辆数：1；是否跟车：None；当前时间：2025-02-16 08:07:15；装货时间：最早2025-02-16 06:00:00，最晚2025-02-16 12:00:00；卸货时间：None；装货地址：四川省成都市金堂县淮口街道金乐路东段1号；卸货地址：四川省雅安市天全县大洼头；装卸次数：一装一卸；是否禁区：None；全程高速：None；价格类型：有价；订金：100.0；订金状态：订金非司机原因可退；价格单位：按趟；出价金额：850.0元；是否油卡支付：None；油卡支付金额：None；支付方式：到付；回单付金额：None；回单邮费

In [6]:
# 数据预处理
logger.info(f"Preprocess_function")
def preprocess_function(examples):
    return teacher_tokenizer(examples, truncation=True, padding="max_length", max_length=512)

logger.info("Preprocessing train dataset")
train_dataset = list(map(preprocess_function, train_dataset))
logger.info("Preprocessing eval dataset")
eval_dataset = list(map(preprocess_function, eval_dataset))


2025-04-24 18:15:05,450 - INFO - Preprocess_function
2025-04-24 18:15:05,451 - INFO - Preprocessing train dataset
2025-04-24 18:15:10,159 - INFO - Preprocessing eval dataset


In [7]:
# 数据收集器
logger.info("DataCollatorForLanguageModeling")
data_collator = DataCollatorForLanguageModeling(tokenizer=teacher_tokenizer, mlm=False)

2025-04-24 18:15:10,413 - INFO - DataCollatorForLanguageModeling


In [8]:
# 定义训练参数
logger.info("Creating trainer")
training_args = TrainingArguments(
    output_dir="./results",            # 训练结果保存路径
    eval_strategy="epoch",             # 每个epoch结束时评估
    learning_rate=5e-5,                # 学习率（默认5e-5是常见选择）
    per_device_train_batch_size=2,     # 每个设备的训练batch size（GPU单卡）
    per_device_eval_batch_size=2,      # 每个设备的评估batch size
    num_train_epochs=3,                # 训练轮次（3轮可能较短，需根据任务调整）
    weight_decay=0.01,                 # 权重衰减（L2正则化）
    logging_dir="./logs",              # 日志保存路径
    logging_steps=100,                 # 每100步记录一次日志
    fp16=False,                        # 是否启用混合精度训练（建议开启）
    gradient_accumulation_steps=4,     # 梯度累积步数（等效batch_size=8）
    report_to="tensorboard",           # 使用TensorBoard记录训练过程
    # distributed_data_parallel=True
    # tensorboard_dir="./tensorboard"  # 可选：指定TensorBoard日志目录
)


2025-04-24 18:15:10,443 - INFO - Creating trainer


In [9]:
# 定义蒸馏配置  weight:添加权重，"loss": "mse"
logger.info("Creating distillation config")
distill_config = DistillationConfig(
    temperature=2.0,  # 温度参数，控制软标签的平滑程度
    hard_label_weight=0.5,  # 真实标签损失权重
    kd_loss_type="ce",      # 知识蒸馏损失类型（交叉熵）
    intermediate_matches=[  # 中间层匹配配置
        {
            "layer_T": 6,    # 教师模型的第6层
            "layer_S": 6,    # 学生模型的第6层
            "feature": "hidden",  # 匹配隐藏层特征
            "weight": 1.0,   # 中间层损失权重
            "loss": "mse"    # 使用均方误差损失
        }
    ]
)


2025-04-24 18:15:11,341 - INFO - Creating distillation config


In [10]:
# 定义训练配置
logger.info("Creating training config")
train_config = TrainingConfig(
    device="cuda" if torch.cuda.is_available() else "cpu",  # 设备选择
    log_dir="./logs",                                     # 日志目录
    output_dir="./outputs"                                # 模型输出目录
    # save_best_model=True,  # 是否保存最佳模型（注释状态）
    # save_last_model=True,  # 是否保存最后模型（注释状态）
    # save_model_every_epoch=True,  # 是否每轮保存模型（注释状态）
    # tensorboard_dir="./tensorboard"  # TensorBoard日志目录（注释状态）
)


2025-04-24 18:15:11,377 - INFO - Creating training config


In [11]:
# 创建蒸馏器
logger.info("Creating distiller")
distiller = GeneralDistiller(
    train_config=train_config,        # 训练配置（包含设备、路径等）
    distill_config=distill_config,    # 蒸馏配置（温度、损失权重等）
    model_T=teacher_model,            # 教师模型
    model_S=student_model,            # 学生模型
    adaptor_T=None,                   # 教师模型适配器（未配置）
    adaptor_S=None                    # 学生模型适配器（未配置）
)


2025-04-24 18:15:11,412 - INFO - Creating distiller


In [12]:
# 开始蒸馏
with distiller:  # 使用蒸馏器上下文管理器，确保资源正确初始化和释放
    logger.info("Starting training")  # 记录训练开始日志

    # 初始化Trainer，集成模型蒸馏配置
    trainer = Trainer(
        model=student_model,  # 学生模型（需要训练的小模型）
        args=training_args,  # 训练参数（如学习率、批次大小、设备等）
        train_dataset=train_dataset,  # 训练数据集（包含输入和标签）
        eval_dataset=eval_dataset,  # 验证数据集（用于评估模型性能）
        data_collator=data_collator,  # 数据批量处理函数（将单条数据组合成批次）
        
        # processing_class=teacher_tokenizer  # 注意：此处可能存在问题（见下方说明）
        # 正确做法：适配器或数据处理逻辑应在蒸馏配置中处理
    )

    # 开始模型训练
    trainer.train()  # 启动训练循环，包含前向传播、损失计算、反向传播等
    trainer.save_model()

    logger.info("Training finished")  # 记录训练结束日志


2025-04-24 18:15:11,445 - INFO - Starting training


Epoch,Training Loss,Validation Loss
1,0.084700,3.578058
2,0.069000,3.884399
3,0.058500,4.245460


2025-04-24 19:01:33,511 - INFO - Training finished
